# TensorFlow 2.0

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%matplotlib inline

## Hyperparameter Tunning

In [2]:
num_epochs = 20
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
train_paths = glob('../dataset/cifar/train/*.png')[:1000]
test_paths = glob('../dataset/cifar/test/*.png')[:1000]

In [15]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [16]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [17]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [18]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [19]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [20]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [21]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

TypeError: in user code:

    <ipython-input-9-6a7a9ccff53d>:2 load_image_label  *
        gfile = tf.io.read_file(path)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\gen_io_ops.py:574 read_file  **
        _, _, _op, _outputs = _op_def_library._apply_op_helper(
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py:492 _apply_op_helper
        raise TypeError("%s expected type of %s." %

    TypeError: Input 'filename' of 'ReadFile' Op has type float32 that does not match expected type of string.


In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

# Learning Rate Scheduler

In [ ]:
import math

In [ ]:
def scheduler(epoch):
    if epoch < 10:
        return 0.001
    else:
        return 0.001 * math.exp(0.1 * (10 - epoch))

learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

## Training

http://localhost:6006

In [26]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[learning_rate_scheduler]
)


Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.
Epoch 1/20
3/3 [==============================] - 2s 562ms/step - loss: 2.1650 - accuracy: 0.1765 - val_loss: 2.2773 - val_accuracy: 0.1458

Epoch 00002: LearningRateScheduler reducing learning rate to 0.001.
Epoch 2/20
3/3 [==============================] - 2s 530ms/step - loss: 2.1468 - accuracy: 0.1765 - val_loss: 2.2784 - val_accuracy: 0.1458

Epoch 00003: LearningRateScheduler reducing learning rate to 0.001.
Epoch 3/20
3/3 [==============================] - 1s 427ms/step - loss: 2.0218 - accuracy: 0.1618 - val_loss: 2.3015 - val_accuracy: 0.1458

Epoch 00004: LearningRateScheduler reducing learning rate to 0.001.
Epoch 4/20
3/3 [==============================] - 2s 538ms/step - loss: 2.2191 - accuracy: 0.2083 - val_loss: 2.2467 - val_accuracy: 0.1354

Epoch 00005: LearningRateScheduler reducing learning rate to 0.001.
Epoch 5/20
3/3 [==============================] - 2s 548ms/step - loss: 2.0890 - accuracy: 0.22

https://www.tensorflow.org/tensorboard/r2/image_summaries#setup